# 7.10. PROJECT. Исследование поведения пользователей - Dano Nikita 

### В этом проекте перед нами поставлена следуюущая задача:
> Вам предстоит проверить два предположения:
> * Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?
> * Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

> Проверку производите на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

### Выполним следующие шаги для решения задачи:
1. Получим данные из файлов 'events.csv' и 'purchase.csv'. Произведем отбор пользователей, зарегистрировавшихся в 2018 году. Преобразуем данные и создадим общий датафрейм.
2. Создадим датафрейм с событиями выбора бесплатного пакета вопросов.
3. Выделим группы пользователей, согласно выбранному уровню сложности.
4. Рассчитаем для каждой группы процент оплат.
5. Найдем разницу во времени между событиями выбора уровня сложности и событиями оплаты, а также среднее время между событиями в группах.
6. Сделаем выводы.

### 1. Получим данные из файлов 'events.csv' и 'purchase.csv'. Произведем отбор пользователей, зарегистрировавшихся в 2018 году. Преобразуем данные и создадим общий датафрейм.

In [24]:
import pandas as pd

events_df = pd.read_csv('events.csv')
eventsf = events_df.loc[(events_df["start_time"] >= "2018-01-01T00:00:00") & (events_df["start_time"] <= "2018-31-12T00:00:00") & (events_df["event_type"] == 'registration')]
registered = eventsf.loc[eventsf["event_type"] == 'registration']["user_id"]

events = events_df[events_df.user_id.isin(registered)]
purchase_df = pd.read_csv('purchase.csv')
purchase = purchase_df[purchase_df.user_id.isin(registered)]
purchase.insert(4, "event_type", "purchase", True)
events = events.rename(columns={'id':'event_id'})
purchase = purchase.rename(columns={'id':'purchase_id'})

purchase_df = total_events_df[total_events_df['event_type'] == 'purchase']
purchase_df = purchase_df[['user_id','event_datetime','amount']].rename(columns={'event_datetime':'purchase_time'})
purchase_df.purchase_time = pd.to_datetime(purchase_df.purchase_time, format='%Y-%m-%dT%H:%M:%S')

total_events_df = pd.concat([events,purchase],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')
total_events_df.start_time = pd.to_datetime(total_events_df.start_time, format='%Y-%m-%dT%H:%M:%S')
total_events_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaN,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaN,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaN,NaN
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaN,NaN
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaN,NaN


### 2. Создадим датафрейм с событиями выбора бесплатного пакета вопросов.

In [25]:
level_choice_df = total_events_df[total_events_df['event_type'] == 'level_choice']
print(level_choice_df['user_id'].value_counts().mean())
level_choice_df = level_choice_df[['user_id','start_time','selected_level']].rename(columns={'start_time':'level_choice_time'})
level_choice_df.head()

1.0


,user_id,level_choice_time,selected_level
19,27835,2018-01-01 20:37:22,medium
23,27839,2018-01-01 22:37:50,hard
36,27840,2018-01-02 05:18:42,medium
38,27845,2018-01-02 06:19:18,hard
45,27842,2018-01-02 08:46:03,easy


### 3. Выделим группы пользователей, согласно выбранному уровню сложности.

In [26]:
level_medium_df = level_choice_df[level_choice_df['selected_level'] == 'medium']
level_medium_users = level_medium_df["user_id"]

level_easy_df = level_choice_df[level_choice_df['selected_level'] == 'easy']
level_easy_users = level_easy_df["user_id"]

level_hard_df = level_choice_df[level_choice_df['selected_level'] == 'hard']
level_hard_users = level_hard_df["user_id"]

level_choice_users = level_choice_df["user_id"]
print("Количество пользователей, выбравших средний уровень сложности: ", len(level_medium_users))
print("Количество пользователей, выбравших низкий уровень сложности: ", len(level_easy_users))
print("Количество пользователей, выбравших высокий уровень сложности: ", len(level_hard_users))
print("Общее количество пользователей: ", len(level_choice_users))

Количество пользователей, выбравших средний уровень сложности:  4645
Количество пользователей, выбравших низкий уровень сложности:  2448
Количество пользователей, выбравших высокий уровень сложности:  1249
Общее количество пользователей:  8342


### 4. Рассчитаем для каждой группы процент оплат.

In [50]:
purchase_df_1 = purchase_df[purchase_df['user_id'].isin(level_medium_users)]
print('-------------------------------------------------------------------------------------------------------------------')
print ('Процент пользователей, выбравших средний уровень, которые оплатили вопросы: {:.2%}'.format(purchase_df_1['user_id'].nunique()/len(level_medium_users)))
print ('Процент пользователей, выбравших средний уровень, которые оплатили вопросы (от общего числа пользователей): {:.2%}'.format(purchase_df_1['user_id'].nunique()/len(level_choice_users)))

purchase_df_2 = purchase_df[purchase_df['user_id'].isin(level_easy_users)]
print('-------------------------------------------------------------------------------------------------------------------')
print ('Процент пользователей, выбравших низкий уровень, которые оплатили вопросы: {:.2%}'.format(purchase_df_2['user_id'].nunique()/len(level_easy_users)))
print ('Процент пользователей, выбравших средний уровень, которые оплатили вопросы (от общего числа пользователей): {:.2%}'.format(purchase_df_2['user_id'].nunique()/len(level_choice_users)))

purchase_df_3 = purchase_df[purchase_df['user_id'].isin(level_hard_users)]
print('-------------------------------------------------------------------------------------------------------------------')
print ('Процент пользователей, выбравших высокий уровень, которые оплатили вопросы: {:.2%}'.format(purchase_df_3['user_id'].nunique()/len(level_hard_users)))
print ('Процент пользователей, выбравших высокий уровень, которые оплатили вопросы (от общего числа пользователей): {:.2%}'.format(purchase_df_3['user_id'].nunique()/len(level_choice_users)))
print('-------------------------------------------------------------------------------------------------------------------')


-------------------------------------------------------------------------------------------------------------------
Процент пользователей, выбравших средний уровень, которые оплатили вопросы: 20.86%
Процент пользователей, выбравших средний уровень, которые оплатили вопросы (от общего числа пользователей): 11.62%
-------------------------------------------------------------------------------------------------------------------
Процент пользователей, выбравших низкий уровень, которые оплатили вопросы: 7.72%
Процент пользователей, выбравших средний уровень, которые оплатили вопросы (от общего числа пользователей): 2.27%
-------------------------------------------------------------------------------------------------------------------
Процент пользователей, выбравших высокий уровень, которые оплатили вопросы: 35.39%
Процент пользователей, выбравших высокий уровень, которые оплатили вопросы (от общего числа пользователей): 5.30%
--------------------------------------------------------------

### 5. Найдем разницу во времени между событиями выбора уровня сложности и событиями оплаты, а также среднее время между событиями в группах.

In [51]:
level_choice_medium_df = level_choice_df[level_choice_df['user_id'].isin(level_medium_users)]
merged_df_1 = level_choice_medium_df.merge(purchase_df_1,on='user_id',how='inner')
merged_df_1['timedelta'] = merged_df_1['purchase_time'] - merged_df_1['level_choice_time']
print('-------------------------------------------------------------------------------------------------------------------')
print("Среднее время между выбором уровня сложности и оплатой в группе пользователей, выбравших средний уровень: ", merged_df_1['timedelta'].mean())


level_choice_easy_df = level_choice_df[level_choice_df['user_id'].isin(level_easy_users)]
merged_df_2 = level_choice_easy_df.merge(purchase_df_2,on='user_id',how='inner')
merged_df_2['timedelta'] = merged_df_2['purchase_time'] - merged_df_2['level_choice_time']
print('-------------------------------------------------------------------------------------------------------------------')
print("Среднее время между выбором уровня сложности и оплатой в группе пользователей, выбравших низкий уровень: ", merged_df_2['timedelta'].mean())


level_choice_hard_df = level_choice_df[level_choice_df['user_id'].isin(level_hard_users)]
merged_df_3 = level_choice_hard_df.merge(purchase_df_3,on='user_id',how='inner')
merged_df_3['timedelta'] = merged_df_3['purchase_time'] - merged_df_3['level_choice_time']
print('-------------------------------------------------------------------------------------------------------------------')
print("Среднее время между выбором уровня сложности и оплатой в группе пользователей, выбравших высокий уровень: ", merged_df_3['timedelta'].mean())
print('-------------------------------------------------------------------------------------------------------------------')


-------------------------------------------------------------------------------------------------------------------
Среднее время между выбором уровня сложности и оплатой в группе пользователей, выбравших средний уровень:  3 days 23:14:13.165118679
-------------------------------------------------------------------------------------------------------------------
Среднее время между выбором уровня сложности и оплатой в группе пользователей, выбравших низкий уровень:  3 days 14:58:52.941798941
-------------------------------------------------------------------------------------------------------------------
Среднее время между выбором уровня сложности и оплатой в группе пользователей, выбравших высокий уровень:  3 days 07:20:41.420814479
-------------------------------------------------------------------------------------------------------------------


### 6. Сделаем выводы.

#### По полученным метрикам видно, что *наименьшие* показатели по проценту оплат среди пользователей выбравших низкий уровень сложности. 
#### Процент оплат от общего числа среди пользователей у пользователей, выбравших средний уровень, выбравших средний уровень  - *наибольший* - это связано с тем, что средний уровень сложности выбирают большинство пользователей.
#### В то же время процент оплат среди пользователей, выбравших высокий уровень - *самый большой*. 
#### Всё это говорит о том, что *уровень вовлечения* выше всего среди пользователей, выбиращих высокий уровень и ниже всего у пользователей, выбирающих низкий уровень.
#### Если подтолкнуть больше пользователей к выбору высокого уровня сложности, то это даст хороший рост процента оплат. Стоит также проработать вопросы низкого уровня сложности - они дают слишком низкий уровень вовлечения.
#### Если говорить о времени между выбором уровня сложности и оплатой - здесь все показатели распределились достаточно равномерно и находятся между 3 и 4 днями в следующей последовательности (от самого короткого к самомму длинному): высокий, низкий и средний уровни сложности. Это опять же подтверждает хорошую вовлеченность при выборе высокого уровня сложности.
